In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from importlib import reload
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input

import os
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
import numpy as np

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")


In [ ]:
num_classes = 2
image_size = 224
batch_size_training = 32
batch_size_validation = 32

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
train_generator = data_generator.flow_from_directory(
    '/content/gdrive/MyDrive/MP/dataset/train',
    target_size = (image_size , image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

valid_generator = data_generator.flow_from_directory(
    '/content/gdrive/MyDrive/MP/dataset/valid',
    target_size = (image_size , image_size),
    batch_size = batch_size_validation,
    class_mode = 'categorical'
    )

Found 12618 images belonging to 2 classes.
Found 156 images belonging to 2 classes.


In [ ]:
model_new = Sequential()

In [ ]:
model_new.add(vgg16.VGG16(include_top = False,pooling='avg',weights='imagenet',))

In [ ]:
model_new.add(Dense(num_classes,activation='softmax'))

In [ ]:
model_new.summary()

In [ ]:
model_new.layers[0].trainable = False
model_new.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
steps_per_epoch_train = len(train_generator)
steps_per_epoch_valid = len(valid_generator)
number_epochs = 3

In [ ]:
fit_history = model_new.fit_generator(train_generator , steps_per_epoch=steps_per_epoch_train,
                                  epochs=number_epochs , verbose=1 , validation_data=valid_generator,
                                  validation_steps=steps_per_epoch_valid)

In [ ]:
model_new.save('/content/gdrive/MyDrive/MP/vgg16_model_new1.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fit_history.history["accuracy"])
plt.plot(fit_history.history['val_accuracy'])
plt.plot(fit_history.history['loss'])
plt.plot(fit_history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
test_generator = data_generator.flow_from_directory('/content/gdrive/MyDrive/MP/dataset/test',target_size = (image_size , image_size),shuffle = False)
steps_per_epoch_test = len(test_generator)

In [ ]:
from keras.models import load_model
vgg16_saved = load_model("/content/gdrive/MyDrive/MP/vgg16_model_new1.h5")

In [ ]:
test_history = vgg16_saved.evaluate(test_generator, steps=steps_per_epoch_test, verbose=1)
print("Accuracy: ",test_history[1])

In [ ]:
predict_vgg16=vgg16_saved.predict_generator(test_generator,steps=steps_per_epoch_test,verbose=1)

In [ ]:
len(predict_vgg16)

In [ ]:
for i in range(0,len(predict_vgg16)):
   Authentic = predict_vgg16[i][0]
   Doctored = predict_vgg16[i][1]
   if(Authentic > Doctored):
     print("Authentic")
   else:
     print("Doctored")  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import image
img = image.load_img("/content/gdrive/MyDrive/MP/samples/8.jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("/content/gdrive/MyDrive/MP/vgg16_model_new1.h5")
output = saved_model.predict(img)
print(output[0][0], output[0][1])
if output[0][0] > output[0][1]:
    print("Authentic")
else:
    print('Doctored')

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import lite

tflite_model = tf.keras.models.load_model('/content/gdrive/MyDrive/MP/vgg16_model_new1.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
open("/content/gdrive/MyDrive/bro/minor_project/vgg16model.tflite", "wb").write(tflite_save)


In [ ]:
import tensorflow as tf
from tensorflow import lite
keras_file=tf.keras.models.load_model('/content/gdrive/MyDrive/MP/vgg16_model_new1.h5')
converter=lite.TFLiteConverter.from_keras_model(keras_file)
tfmodel=converter.convert()
open("/content/gdrive/MyDrive/bro/minor_project/vgg16_maruti.tflite",'wb').write(tfmodel)